# Compute reference probabilities with Monte-Carlo

In this example, we perform a reliability benchmark based on a large Monte-Carlo sample. In order to limit the elapsed time, we consider a limited elapsed time for each problem. In order to get the best possible accuracy within this time limit, we set the coefficient of variation to zero.

In [1]:
import openturns as ot
import otbenchmark as otb
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

In [2]:
problemslist = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(problemslist)
numberOfProblems

26

In [3]:
coefficientOfVariation = 0.0
maximumOuterSampling = 10 ** 5
blockSize = 10 ** 0 # 10 ** 4 for real simulations
blockSize

10000

In [4]:
confidenceLevel = 0.95
maximumDurationSeconds = 60.0

In [5]:
model_names = [problemslist[i].getName() for i in range(numberOfProblems)]
metrics = ["PF", "N. function calls", "PMin", "PMax", "C.O.V.", "Digits", "Time (s)"]
resultArray = np.zeros((numberOfProblems, len(metrics)))
for i in tqdm(range(numberOfProblems)):
    startTime = time.time()
    problem = problemslist[i]
    name = problem.getName()
    event = problem.getEvent()
    g = event.getFunction()
    factory = otb.ProbabilitySimulationAlgorithmFactory()
    algo = factory.buildMonteCarlo(problem)
    algo.setMaximumOuterSampling(maximumOuterSampling)
    algo.setBlockSize(blockSize)
    algo.setMaximumCoefficientOfVariation(coefficientOfVariation)
    timer = ot.TimerCallback(maximumDurationSeconds)
    algo.setStopCallback(timer)
    initialNumberOfCall = g.getEvaluationCallsNumber()
    algo.run()
    result = algo.getResult()
    numberOfFunctionEvaluations = g.getEvaluationCallsNumber() - initialNumberOfCall
    computedProbability = result.getProbabilityEstimate()
    confidenceLength = result.getConfidenceLength(confidenceLevel)
    pmin = computedProbability - 0.5 * confidenceLength
    pmax = computedProbability + 0.5 * confidenceLength
    cov = result.getCoefficientOfVariation()
    if cov > 0.0:
        expectedDigits = -np.log10(cov) - 1.0
    else:
        expectedDigits = 0.0
    stopTime = time.time()
    elapsedTime = stopTime - startTime
    resultArray[i][0] = computedProbability
    resultArray[i][1] = numberOfFunctionEvaluations
    resultArray[i][2] = pmin
    resultArray[i][3] = pmax
    resultArray[i][4] = cov
    resultArray[i][5] = expectedDigits
    resultArray[i][6] = elapsedTime

100%|██████████| 26/26 [26:00<00:00, 60.00s/it]


In [6]:
df = pd.DataFrame(resultArray, index=model_names, columns=metrics)
df

,PF,N. function calls,PMin,PMax,C.O.V.,Digits,Time (s)
RP8,7.895145e-04,39960000.0,7.808056e-04,7.982235e-04,0.005628,1.249643,60.007233
RP14,7.743634e-04,149240000.0,7.699004e-04,7.788265e-04,0.002941,1.531562,60.001761
RP22,4.206241e-03,332970000.0,4.199289e-03,4.213193e-03,0.000843,2.074043,60.000491
RP24,2.854640e-03,333180000.0,2.848911e-03,2.860369e-03,0.001024,1.989714,60.000430
RP25,4.112751e-05,301550000.0,4.040366e-05,4.185136e-05,0.008980,1.046733,60.000382
RP28,1.688429e-07,355360000.0,1.261184e-07,2.115674e-07,0.129106,-0.110946,60.001737
RP31,3.224597e-03,336910000.0,3.218543e-03,3.230651e-03,0.000958,2.018674,60.001431
RP33,2.575854e-03,288490000.0,2.570004e-03,2.581703e-03,0.001159,1.936064,60.000849
RP35,3.480717e-03,277910000.0,3.473792e-03,3.487641e-03,0.001015,1.993522,60.000416
RP38,8.051600e-03,156590000.0,8.037601e-03,8.065598e-03,0.000887,2.052057,60.001410


In [7]:
df.to_csv('reliability_compute_reference_proba.csv')